In [1]:
import numpy as np

In [8]:
# Generate cost graph
def cost_graph(v=10, f=0.5):
    graph = np.full((v,v), np.inf)
    
    for i in range(len(graph)):
        for j in range(len(graph[i])):
            # When not a diagonal element, i.e no self loop
            if i != j:
                # Randomly set values to graph replaces np.inf
                if np.random.rand() < f:
                    graph[i][j] = np.random.randint(1,20)
    return graph

# Generate Pheromone graph
def pheromone_graph(v=10):
    pheromone = np.ones((v,v))
    return pheromone


# Selecting next vertex in a path to reach destination
def selectNextVectex(currVertex, graph, pheromone):
    alpha, beta = 1, 1 # Controlling the influence of pheromone and path length
    
    # Cost of moving to other vertex from current vertex
    possible_pathCost = np.power(1/graph[currVertex], beta)
    
    # Existing pheromone levels at other vertex from current vertex
    possible_pheromone = np.power(pheromone[currVertex], alpha)
    
    # Probability of moving to other vector from current vertex
    P = (possible_pathCost * possible_pheromone)/ np.dot(possible_pathCost, possible_pheromone)
    
    # Cumulative sum - for roulette wheel based path selection
    C = np.cumsum(P)
    
    # Roulette wheel selection
    r = np.random.rand()
    selected_vertex = np.where(C > r)[0][0] # Selecting the first vertex that satisfies the condition 
    
    return selected_vertex


# Finding all paths and their corresponding costs for a generation
def generationOfAnts(graph, pheromone, size=20, start=0, end=6):
    paths = []
    costs = []

    for i in range(size):
        path = [start]
        j = 0
        cost = 0
        prev = start
        
        while 1:
            next_vertex = selectNextVectex(prev, graph, pheromone)
            while next_vertex == prev:
                next_vertex = selectNextVectex(prev, graph, pheromone)
                
            path.append(next_vertex)
            cost += graph[prev][next_vertex]
            if next_vertex == end:
                break
            prev = next_vertex
            j += 1
        paths.append(np.array(path))
        costs.append(cost)

    return paths, costs


# Update pheromone level on paths after each generation 
## This implementation is not 'the most' accurate. Because pheromone level on paths ideally must be updated
## after each ant movement. But here it is updated only after a generation (10 ants = 1 generation)
def updatePheromones(paths, costs, pheromone, decay=0.5):
    # pheromone secreted by ant based on the quality of path's cost
    pheromone_secreted = 1/np.array(costs)
    
    # Vaporization in existing pheromone
    pheromone = (1-decay) * pheromone
    
    # Adding the newly secreated pheromone to path
    for i in range(len(paths)):
        # Ant i's path
        path = paths[i]
        for j in range(len(path)-1):
            # Adding ant i's pheromone to its entire path
            pheromone[path[j]][path[j+1]] += pheromone_secreted[i] 
    return pheromone
        


In [9]:
graph = cost_graph(v=10, f=0.5)
graph

array([[inf, inf, inf,  9.,  2., inf, inf, inf,  1., inf],
       [ 5., inf, inf, inf, 12., inf, inf,  2.,  4., 18.],
       [inf,  4., inf, inf, inf, 14., inf,  6., inf,  9.],
       [ 4., inf, inf, inf, inf, inf, 11.,  1., 15.,  9.],
       [inf, 16., inf, inf, inf, 10.,  8., 11., inf, inf],
       [inf,  3., inf, inf, inf, inf, inf,  6.,  4., inf],
       [ 8., inf, inf, 13., inf, 10., inf, inf,  6.,  8.],
       [inf, inf, inf,  7., 13.,  9.,  6., inf,  6., inf],
       [inf, 18.,  7.,  5.,  8.,  1., inf, 15., inf, inf],
       [ 4., inf, inf, inf,  9., 19., inf, inf,  8., inf]])

In [10]:
pheromone = pheromone_graph(v=10)
pheromone

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [11]:
max_iter = 100
threshold = 9
for i in range(max_iter):
    print(f'Iteration: {i+1}')
    paths, costs = generationOfAnts(graph, pheromone,start=0, end=9, size=10) # 10 ants = 10 paths
    pheromone = updatePheromones(paths, costs, pheromone)
    
    # When several paths(set a threshold count) has same cost --> Optimal path has been found, then terminate
    unique_cost, counts = np.unique(costs, return_counts=True)
    print(unique_cost, counts)
    print('--------------------------------------------')
    
    if counts[0] > threshold:
        break
print(paths)

    


Iteration: 1
[ 22.  28.  52.  54.  75.  95.  98. 158. 217. 246.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 2
[ 23.  36.  53.  76. 100. 111. 117. 150. 165. 200.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 3
[ 63. 114. 122. 136. 140. 200. 261. 270. 301. 403.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 4
[ 21.  65.  91. 124. 142. 185. 198. 217. 354. 392.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 5
[ 62.  68.  75.  81.  87.  93. 112. 145. 152. 286.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 6
[ 75. 105. 109. 117. 127. 181. 202. 225. 264. 289.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 7
[ 22.  79. 106. 108. 112. 149. 156. 208. 223. 348.] [1 1 1 1 1 1 1 1 1 1]
--------------------------------------------
Iteration: 8
[ 74.  85.  86. 105. 109. 110. 178. 207. 209. 336.] [1 1 1 1 1 

## Explanation of final iteration loop
- On each iteration the list of unique cost of all the paths are returned along with their count.
- The cost and its count are returned in increasing order of sorting. So 0th index of `unique_cost` is minimum and 0th index of `counts` variable is the count of the minimum cost 
- Our aim at subsequent iteration is to increase the count of minimum cost.
- After 26 iterations, all paths became same and returns minimum cost as 21
- Finally the list of all paths is printed. So you can see, after 26th iteration all ants are following same path. Which implies pheromone level at that particular path is more than any other path.